## Transfer Learning

1. Perform classification on FashionMNIST, fashion apparels dataset, using a pretrained model which is trained on MNIST handwritten digit classification dataset.

In [57]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms as T
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST

In [58]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(128, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.classification_head = nn.Sequential(
            nn.Linear(64, 20, bias=True),
            nn.ReLU(),
            nn.Linear(20, 10, bias=True)
        )            
        
    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(features.shape[0], -1))

In [59]:
mnist_testset = datasets.FashionMNIST(root='./data', train= False, download=True, transform=ToTensor())

test_loader = DataLoader(mnist_testset, batch_size=50, shuffle=True)

In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [61]:
model=CNNClassifier()
model = torch.load('./models/mnist_model.pt')
model.to(device)

CNNClassifier(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification_head): Sequential(
    (0): Linear(in_features=64, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
  )
)

In [62]:
print("Model's state dict")
for param_tensor in model.state_dict().keys():
    print(param_tensor, '\t', model.state_dict()[param_tensor].size())

Model's state dict
net.0.weight 	 torch.Size([64, 1, 3, 3])
net.0.bias 	 torch.Size([64])
net.3.weight 	 torch.Size([128, 64, 3, 3])
net.3.bias 	 torch.Size([128])
net.6.weight 	 torch.Size([64, 128, 3, 3])
net.6.bias 	 torch.Size([64])
classification_head.0.weight 	 torch.Size([20, 64])
classification_head.0.bias 	 torch.Size([20])
classification_head.2.weight 	 torch.Size([10, 20])
classification_head.2.bias 	 torch.Size([10])


In [63]:
model.eval()

CNNClassifier(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification_head): Sequential(
    (0): Linear(in_features=64, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
  )
)

In [64]:
correct = 0
total = 0
for i, vdata in enumerate(test_loader):
    tinputs, tlabels = vdata
    tinputs = tinputs.to(device)
    tlabels = tlabels.to(device)
    toutputs = model(tinputs)
    _,predicted = torch.max(toutputs,1)
    print('True label: {}'.format(tlabels))
    print('Predicted label: {}'.format(predicted))
    total += tlabels.size(0)
    
    correct += (predicted == tlabels).sum()
accuracy = 100.0 * correct / total
print('Accuracy : {}'.format(accuracy))

True label: tensor([6, 8, 2, 2, 9, 1, 6, 7, 8, 4, 7, 2, 6, 5, 3, 8, 9, 9, 6, 8, 0, 5, 1, 3,
        5, 9, 8, 1, 6, 9, 4, 7, 9, 0, 0, 8, 1, 4, 8, 7, 1, 1, 2, 9, 7, 9, 2, 7,
        3, 2])
Predicted label: tensor([0, 0, 0, 0, 3, 8, 0, 6, 0, 0, 4, 0, 0, 5, 1, 0, 3, 2, 0, 0, 2, 4, 8, 8,
        2, 8, 6, 8, 0, 2, 0, 4, 2, 8, 8, 6, 8, 2, 6, 4, 0, 8, 8, 8, 4, 0, 1, 3,
        8, 0])
True label: tensor([0, 9, 5, 2, 1, 1, 8, 2, 5, 3, 5, 0, 8, 6, 3, 2, 7, 2, 1, 4, 1, 8, 3, 1,
        3, 5, 1, 3, 5, 5, 2, 9, 6, 2, 1, 8, 3, 7, 0, 5, 1, 9, 6, 3, 1, 6, 0, 3,
        4, 0])
Predicted label: tensor([8, 1, 4, 2, 0, 0, 6, 0, 6, 6, 0, 8, 0, 0, 8, 0, 2, 6, 8, 0, 0, 6, 8, 8,
        1, 4, 8, 8, 5, 0, 0, 8, 0, 0, 8, 8, 8, 6, 8, 9, 8, 2, 0, 0, 8, 0, 8, 8,
        0, 0])
True label: tensor([2, 7, 1, 9, 2, 9, 8, 8, 1, 0, 7, 0, 2, 5, 1, 2, 7, 6, 1, 1, 6, 1, 1, 1,
        8, 1, 5, 9, 4, 3, 1, 9, 1, 9, 0, 7, 3, 6, 8, 9, 1, 6, 4, 1, 3, 0, 4, 9,
        0, 4])
Predicted label: tensor([6, 6, 8, 3, 0, 3, 4, 6, 8, 8, 

2. Learn the AlexNet architecture and apply transfer learning to perform the classification 
task. Using the pre-trained AlexNet, classify images from the cats_and_dogs_filtered 
dataset downloaded from the below link. Finetune the classifier given in AlexNet as a two class classifier. Perform pre-processing of images as per the requirement.

In [4]:
import PIL.Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import glob
import torchvision
from torchvision import transforms
from torchvision.models import AlexNet_Weights
import torchvision.datasets as datasets
import pandas as pd

In [6]:
#model = torchvision.models.alexnet()
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)

Using cache found in /Users/harryraj/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/harryraj/Desktop/Sem VI labs/DL Lab/pythonProject/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/harryraj/Desktop/Sem VI labs/DL Lab/pythonProject/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
def get_df(path, classes=['dogs', 'cats']):
    paths = pd.DataFrame({'class': [], 'path': []})
    for c in classes:
        df = pd.DataFrame({
            'class': c,
            'path': glob.glob(path + c + '/*')
        })
        paths = pd.concat([paths, df])
    paths.reset_index(inplace=False)
    return paths

In [8]:
class MyDataset(Dataset):
    def __init__(self, df, classes, transform=None):
        self.paths = df
        self.classes = classes
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        row = self.paths.iloc[idx]
        img = Image.open(row['path'])
        if self.transform is not None:
            return self.transform(img), self.classes[row['class']]
        else:
            return img, self.classes[row['class']]

In [9]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
        train_losses.append(loss.item())
        train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [10]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
EPOCHS = 10
BATCH_SIZE_TRAIN = 16
BATCH_SIZE_TEST = 10
LR = 0.001
LOG_INTERVAL = 100
RANDOM_SEED = 1
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cpu")
TRANSFORM = T.Compose([
    T.ToTensor(),
    T.Resize([224, 224]),
])

CLASSES = {'dogs': 0, 'cats': 1}

torch.manual_seed(RANDOM_SEED)

In [12]:
train_dataset = MyDataset(get_df('cats_and_dogs_filtered/train/'), CLASSES, TRANSFORM)
test_dataset = MyDataset(get_df('cats_and_dogs_filtered/validation/'), CLASSES, TRANSFORM)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE_TEST, shuffle=True)

In [13]:
for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    *model.classifier[:-1],
    nn.Linear(4096, 2, bias=True)
)

model = model.to(DEVICE)
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(EPOCHS + 1)]
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/2000 (0%)]	Loss: 0.693097
Train Epoch: 1 [1600/2000 (80%)]	Loss: 0.382577

Test set: Avg. loss: 0.0281, Accuracy: 889/1000 (89%)
Train Epoch: 2 [0/2000 (0%)]	Loss: 0.322746
Train Epoch: 2 [1600/2000 (80%)]	Loss: 0.196349

Test set: Avg. loss: 0.0251, Accuracy: 896/1000 (90%)

Train Epoch: 3 [0/2000 (0%)]	Loss: 0.106392
Train Epoch: 3 [1600/2000 (80%)]	Loss: 0.288080

Test set: Avg. loss: 0.0240, Accuracy: 899/1000 (90%)

Train Epoch: 4 [0/2000 (0%)]	Loss: 0.325195
Train Epoch: 4 [1600/2000 (80%)]	Loss: 0.213588

Test set: Avg. loss: 0.0228, Accuracy: 908/1000 (91%)

Train Epoch: 5 [0/2000 (0%)]	Loss: 0.211119
Train Epoch: 5 [1600/2000 (80%)]	Loss: 0.180716

Test set: Avg. loss: 0.0219, Accuracy: 907/1000 (91%)

Train Epoch: 6 [0/2000 (0%)]	Loss: 0.130455
Train Epoch: 6 [1600/2000 (80%)]	Loss: 0.419849

Test set: Avg. loss: 0.0217, Accuracy: 912/1000 (91%)

Train Epoch: 7 [0/2000 (0%)]	Loss: 0.117483
Train Epoch: 7 [1600/2000 (80%)]	Loss: 0.293264

Test set: Avg. loss:

3.Implement check points in PyTorch by saving model state_dict, optimizer state_dict, epochs 
and loss during training so that the training can be resumed at a later point. Also, illustrate 
the use of check point to save the best found parameters during training.

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2,2), stride=2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2,2), stride=2),
            nn.Conv2d(128, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )
        self.classification_head = nn.Sequential(
            nn.Linear(64, 20, bias=True),
            nn.ReLU(),
            nn.Linear(20, 10, bias=True)
        )

    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(x.size(0), -1))

# Create the dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())

batch_size = 4
train_data_loader = DataLoader(mnist_trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_testset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNNClassifier().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

EPOCHS = 10

# Function to train one epoch
def train_one_epoch(epoch_index):
    total_loss = 0.
    for i, data in enumerate(train_data_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / (len(train_data_loader) * batch_size)

# Training loop with checkpointing every 2 epochs
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))
    model.train(True)
    avg_loss = train_one_epoch(epoch)
    print('LOSS train {}'.format(avg_loss))
    
    if (epoch + 1) % 2 == 0:
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss
        }
        torch.save(checkpoint, f'checkpoint_epoch_{epoch + 1}.pt')

# Resuming training from the checkpoint
checkpoint = torch.load('checkpoint_epoch_2.pt')  # Load the checkpoint
model.load_state_dict(checkpoint['model_state_dict'])  # Load model state
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])  # Load optimizer state
epoch = checkpoint['epoch']  # Load epoch
loss = checkpoint['loss']  # Load loss

# Continue training from the next epoch
for epoch in range(epoch, EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))
    model.train(True)
    avg_loss = train_one_epoch(epoch)
    print('LOSS train {}'.format(avg_loss))
    
    if (epoch + 1) % 2 == 0:
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss
        }
        torch.save(checkpoint, f'checkpoint_epoch_{epoch + 1}.pt')


CNNClassifier(
  (net): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification_head): Sequential(
    (0): Linear(in_features=64, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
  )
)
EPOCH 1:
LOSS train 0.4121341368558041
EPOCH 2:
LOSS train 0.07600859776814468
EPOCH 3:
LOSS train 0.04769954553960366
EPOCH 4:
LOSS train 0.03719800033136016
EPOCH 5:
LOSS train 0.031263634294604464
EPOCH 6:
LOSS train 0.027118428241219135
EPOCH 7:
LOSS train 0.024046691821841704
EPOCH 